# Project 4 - West Nile Virus Prediction

## Problem Statement
The intent of this project is to analyze weather data and GIS data and predicting the presence of the West Nile virus, for a given time, location, and species. 

##  General Approach
- [Data cleaning and imputation](#Data-Cleaning)
- [Data visualization](#Data-Visualization)
- [Feature selection](#Featuion-Selection)
- [Cross validation](#Cross-Validation)
- [Model fitting](#Model-Fitting)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from haversine import haversine

In [2]:
# Load dataset 
train = pd.read_csv('assets/train.csv')
test = pd.read_csv('assets/test.csv')
weather = pd.read_csv('assets/weather.csv')
spray = pd.read_csv('assets/spray.csv')

### Data Cleaning and Imputation

#### Weather data

In [3]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


We will look through the weather dataset to see if we can reasonably fill the missing values and drop columns which we feel are less or not relevant to help us predict the presence of the mosquito or virus.

In [4]:
# These are the columns we identified as low variance 
columns_to_drop = ['Depart', 'Sunrise', 'Sunset','Depth', 'Water1', 'SnowFall', 'Heat', 'Cool']

In [5]:
weather.drop(columns=columns_to_drop, inplace=True)

In [6]:
weather.dtypes

Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            object
DewPoint         int64
WetBulb         object
CodeSum         object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object

In [7]:
# Convert Date object to datetime format
weather['Date'] = pd.to_datetime(weather['Date'])

In [8]:
# obtaining Tavg from Tmin and Tmax
weather['Tavg'] = (weather['Tmax'] + weather['Tmin']) / 2
weather['Tavg'] = weather['Tavg'].astype(float)

In [9]:
weather.shape

(2944, 14)

In [10]:
# Check missing values of WetBulb
weather[weather['WetBulb'] == 'M']

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
848,1,2009-06-26,86,69,77.5,60,M,,0.00,M,29.85,6.4,4,8.2
2410,1,2013-08-10,81,64,72.5,57,M,,0.00,M,30.08,5.3,5,6.5
2412,1,2013-08-11,81,60,70.5,61,M,RA,0.01,29.35,30.07,2.0,27,3.0
2415,2,2013-08-12,85,69,77.0,63,M,RA,0.66,29.27,29.92,4.5,26,7.7


In [11]:
# since the value of DewPoint is similar for both stations on the same day, 
# we will replace the missing values with values from the same day. 
weather.loc[[848, 849, 2410, 2411, 2412, 2413, 2414, 2415], :]

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
848,1,2009-06-26,86,69,77.5,60,M,,0.00,M,29.85,6.4,4,8.2
849,2,2009-06-26,86,72,79.0,61,67,,0.00,29.20,29.83,6.4,4,8.0
2410,1,2013-08-10,81,64,72.5,57,M,,0.00,M,30.08,5.3,5,6.5
2411,2,2013-08-10,81,68,74.5,55,63,,0.00,M,30.07,6.0,6,7.4
2412,1,2013-08-11,81,60,70.5,61,M,RA,0.01,29.35,30.07,2.0,27,3.0
2413,2,2013-08-11,84,63,73.5,57,64,,T,29.42,30.06,4.0,24,5.4
2414,1,2013-08-12,82,67,74.5,65,68,RA DZ,0.27,29.21,29.93,3.5,27,7.5
2415,2,2013-08-12,85,69,77.0,63,M,RA,0.66,29.27,29.92,4.5,26,7.7


In [12]:
# replace the values accordingly, converting to int
weather.at[848, 'WetBulb'] = weather.at[849, 'WetBulb'] 
weather.at[2410, 'WetBulb'] = weather.at[2411, 'WetBulb'] 
weather.at[2412, 'WetBulb'] = weather.at[2413, 'WetBulb'] 
weather.at[2415, 'WetBulb'] = weather.at[2414, 'WetBulb'] 
weather['WetBulb'] = weather['WetBulb'].astype(int);

In [13]:
# Check missing values of StnPressure
weather[weather['StnPressure'] == 'M']

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
87,2,2007-06-13,86,68,77.0,53,62,,0.00,M,M,7.0,5,M
848,1,2009-06-26,86,69,77.5,60,67,,0.00,M,29.85,6.4,4,8.2
2410,1,2013-08-10,81,64,72.5,57,63,,0.00,M,30.08,5.3,5,6.5
2411,2,2013-08-10,81,68,74.5,55,63,,0.00,M,30.07,6.0,6,7.4


In [14]:
# since the value of StnPressure is similar for both stations on the same day, 
# we will replace the missing values with values from the same day. 
# we will replace the rows 2410 and 2411 with the column mean as both stations have missing data
weather.loc[[86, 87, 848, 849, 2410, 2411], :]

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
86,1,2007-06-13,87,60,73.5,53,62,,0.00,29.36,30.09,7.2,5,8.6
87,2,2007-06-13,86,68,77.0,53,62,,0.00,M,M,7.0,5,M
848,1,2009-06-26,86,69,77.5,60,67,,0.00,M,29.85,6.4,4,8.2
849,2,2009-06-26,86,72,79.0,61,67,,0.00,29.20,29.83,6.4,4,8.0
2410,1,2013-08-10,81,64,72.5,57,63,,0.00,M,30.08,5.3,5,6.5
2411,2,2013-08-10,81,68,74.5,55,63,,0.00,M,30.07,6.0,6,7.4


In [15]:
# replace the values accordingly, converting series to float
weather.at[87, 'StnPressure'] = weather.at[86, 'StnPressure'] 
weather.at[848, 'StnPressure'] = weather.at[849, 'StnPressure']
weather.at[2410, 'StnPressure'] = np.nan
weather.at[2411, 'StnPressure'] = np.nan
weather['StnPressure'].fillna(weather['StnPressure'].astype(float).mean(), inplace=True)
weather['StnPressure'] = weather['StnPressure'].astype(float);

In [16]:
# Check missing values of SeaLevel
weather[weather['SeaLevel'] == 'M']

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
87,2,2007-06-13,86,68,77.0,53,62,,0.00,29.36,M,7.0,5,M
832,1,2009-06-18,80,61,70.5,63,67,RA BR,0.12,29.08,M,6.7,16,7.9
994,1,2009-09-07,77,59,68.0,59,62,BR,0.00,29.39,M,5.8,3,4.0
1732,1,2011-09-08,75,57,66.0,53,59,RA,T,29.34,M,13.0,2,13.4
1745,2,2011-09-14,60,48,54.0,45,51,RA BR HZ FU,T,29.47,M,6.0,32,M
1756,1,2011-09-20,74,49,61.5,54,58,MIFG BCFG BR,0.00,29.26,M,7.3,18,7.3
2067,2,2012-08-22,84,72,78.0,51,61,,0.00,29.39,M,4.7,19,M
2090,1,2012-09-03,88,71,79.5,70,73,BR,0.00,29.17,M,4.6,6,4.4
2743,2,2014-07-23,76,64,70.0,56,61,,0.00,29.47,M,16.4,2,16.7


In [17]:
# since the value of SeaLevel is similar for both stations on the same day, 
# we will replace the missing values with values from the same day. 
weather.loc[[86, 87, 832, 833, 994, 995, 1732, 1733, 1744, 1745, 1756, 1757, 2066, 2067, 2090, 2091, 2742, 2743], :]

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
86,1,2007-06-13,87,60,73.5,53,62,,0.00,29.36,30.09,7.2,5,8.6
87,2,2007-06-13,86,68,77.0,53,62,,0.00,29.36,M,7.0,5,M
832,1,2009-06-18,80,61,70.5,63,67,RA BR,0.12,29.08,M,6.7,16,7.9
833,2,2009-06-18,81,63,72.0,64,67,TSRA BR HZ,0.11,29.15,29.79,3.7,17,5.8
994,1,2009-09-07,77,59,68.0,59,62,BR,0.00,29.39,M,5.8,3,4.0
995,2,2009-09-07,77,63,70.0,59,63,BR HZ,0.00,29.44,30.09,6.3,4,6.9
1732,1,2011-09-08,75,57,66.0,53,59,RA,T,29.34,M,13.0,2,13.4
1733,2,2011-09-08,74,62,68.0,54,59,RA DZ BR,0.06,29.36,30.03,14.9,2,15.2
1744,1,2011-09-14,58,47,52.5,43,49,RA BR HZ FU,0.08,29.39,30.09,6.3,34,7.3
1745,2,2011-09-14,60,48,54.0,45,51,RA BR HZ FU,T,29.47,M,6.0,32,M


In [18]:
# replace the values accordingly, converting series to float
weather.at[87, 'SeaLevel'] = weather.at[86, 'SeaLevel'] 
weather.at[832, 'SeaLevel'] = weather.at[833, 'SeaLevel'] 
weather.at[994, 'SeaLevel'] = weather.at[995, 'SeaLevel']
weather.at[1732, 'SeaLevel'] = weather.at[1733, 'SeaLevel'] 
weather.at[1745, 'SeaLevel'] = weather.at[1744, 'SeaLevel'] 
weather.at[1756, 'SeaLevel'] = weather.at[1757, 'SeaLevel'] 
weather.at[2067, 'SeaLevel'] = weather.at[2066, 'SeaLevel'] 
weather.at[2090, 'SeaLevel'] = weather.at[2091, 'SeaLevel'] 
weather.at[2743, 'SeaLevel'] = weather.at[2742, 'SeaLevel'] 
weather['SeaLevel'] = weather['SeaLevel'].astype(float);

In [19]:
# since precipitation seems promotes breeding of mosquitos we will check for 'RA/DZ/TS' for occurance of rain
# we will create another column, 'Rain' and indicate 1 for 'RA/DZ/TS' and 0 for other codes
weather['Rain'] = weather['CodeSum'].apply(lambda x : 1 if 'RA' in x else (1 if 'DZ' in x else (1 if 'TS' in x else 0)))


In [20]:
# we will then drop 'CodeSum'
weather.drop(columns='CodeSum', inplace=True)

In [21]:
# verifying columns and datatypes
weather.dtypes

Station                 int64
Date           datetime64[ns]
Tmax                    int64
Tmin                    int64
Tavg                  float64
DewPoint                int64
WetBulb                 int64
PrecipTotal            object
StnPressure           float64
SeaLevel              float64
ResultSpeed           float64
ResultDir               int64
AvgSpeed               object
Rain                    int64
dtype: object

In [22]:
weather = weather[weather['AvgSpeed'] != 'M']

In [23]:
weather['AvgSpeed'] = weather['AvgSpeed'].astype(float)

In [24]:
weather['PrecipTotal'].value_counts()

0.00    1575
  T      317
0.01     127
0.02      63
0.03      46
        ... 
1.49       1
1.96       1
1.04       1
1.73       1
1.24       1
Name: PrecipTotal, Length: 168, dtype: int64

In [25]:
weather['PrecipTotal'] = weather['PrecipTotal'].map(lambda x: '0.00' if 'M' in x else x)
weather['PrecipTotal'] = weather['PrecipTotal'].map(lambda x: '0.00' if 'T' in x else x)

In [26]:
# convert to floating dtype
weather['PrecipTotal'] = weather['PrecipTotal'].astype(float)

In [27]:
weather.dtypes

Station                 int64
Date           datetime64[ns]
Tmax                    int64
Tmin                    int64
Tavg                  float64
DewPoint                int64
WetBulb                 int64
PrecipTotal           float64
StnPressure           float64
SeaLevel              float64
ResultSpeed           float64
ResultDir               int64
AvgSpeed              float64
Rain                    int64
dtype: object

In [28]:
# Final weather df
weather.head(10)

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Rain
0,1,2007-05-01,83,50,66.5,51,56,0.0,29.10,29.82,1.7,27,9.2,0
1,2,2007-05-01,84,52,68.0,51,57,0.0,29.18,29.82,2.7,25,9.6,0
2,1,2007-05-02,59,42,50.5,42,47,0.0,29.38,30.09,13.0,4,13.4,0
3,2,2007-05-02,60,43,51.5,42,47,0.0,29.44,30.08,13.3,2,13.4,0
4,1,2007-05-03,66,46,56.0,40,48,0.0,29.39,30.12,11.7,7,11.9,0
5,2,2007-05-03,67,48,57.5,40,50,0.0,29.46,30.12,12.9,6,13.2,0
6,1,2007-05-04,66,49,57.5,41,50,0.0,29.31,30.05,10.4,8,10.8,1
7,2,2007-05-04,78,51,64.5,42,50,0.0,29.36,30.04,10.1,7,10.4,0
8,1,2007-05-05,66,53,59.5,38,49,0.0,29.40,30.10,11.7,7,12.0,0
9,2,2007-05-05,66,54,60.0,39,50,0.0,29.46,30.09,11.2,7,11.5,0


#### Train data

In [29]:
def closest_point(point):
    station1, station2 = [41.995, -87.933], [41.786, -87.752]  # Fixed coordinates for two stations
    points = [station1, station2] 
    if cdist([point], points).argmin() == 0: return 1 # return index of closest point
    return 2

In [30]:
# Assign Station to train data based on station coordinates
train['Station'] = [closest_point(x) for x in train[['Latitude','Longitude']].values]
test['Station'] = [closest_point(x) for x in test[['Latitude','Longitude']].values]

In [31]:
# Convert test and train data to datetime format
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

### Data Visualization

### Feature Engineering

In [32]:
weather = weather.set_index('Date')

In [33]:
for c in ['PrecipTotal','Tavg','Tmin','Tmax','DewPoint']:
    for p in [14,28,60]:
        if c == 'PrecipTotal':
            weather[f'{c}_{str(p)}'] = weather[c].rolling(p, min_periods=1).sum()
        elif c == 'Tmin':
            weather[f'{c}_{str(p)}'] = weather[c].rolling(p, min_periods=1).min()
        elif c == 'Tmax':
            weather[f'{c}_{str(p)}'] = weather[c].rolling(p, min_periods=1).max()
        else:
            weather[f'{c}_{str(p)}'] = weather[c].rolling(p, min_periods=1).mean()

### Merge weather and train data

In [34]:
train = train.merge(weather, on=['Date','Station'])

In [35]:
df = pd.get_dummies(train, columns=['Species'])

In [36]:
df['Month'] = df['Date'].dt.month
df["Day"] = df['Date'].dt.dayofyear

In [37]:
# Find out coordinates with the highest WnvPresent
train[train['WnvPresent'] == 1]['Latitude'].value_counts().head()


41.974689    66
41.673408    41
41.954690    18
41.726465    16
41.964242    14
Name: Latitude, dtype: int64

In [38]:
# Find out coordinates with the highest WnvPresent
train[train['WnvPresent'] == 1]['Longitude'].value_counts().head()


-87.890615    66
-87.599862    41
-87.800991    18
-87.585413    16
-87.757639    14
Name: Longitude, dtype: int64

In [39]:
# Get the top 5 latitude and longitude coordinates with wnvpresent area
wnvpresent_lat_index = train[train['WnvPresent'] == 1]['Latitude'].value_counts().index[:2]
wnvpresent_lon_index = train[train['WnvPresent'] == 1]['Longitude'].value_counts().index[:2]

In [40]:
wnvpresent_lat_lon = [(lat,lon) for lat,lon in zip(wnvpresent_lat_index,wnvpresent_lon_index)]
wnvpresent_lat_lon # Top 5 lat, lon areas with wnvpresent

[(41.974689, -87.890615), (41.673408, -87.599862)]

In [41]:
# Calculate the haversine distance from train data to coordiates with wnv persent area
for i,area in enumerate(wnvpresent_lat_lon):
    df['Dist_' + str(i)] = [haversine(row, (area),unit='mi') for row in zip(train['Latitude'],train['Longitude'])]



In [42]:
df._get_numeric_data().columns

Index(['Block', 'Latitude', 'Longitude', 'AddressAccuracy', 'NumMosquitos',
       'WnvPresent', 'Station', 'Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb',
       'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir',
       'AvgSpeed', 'Rain', 'PrecipTotal_14', 'PrecipTotal_28',
       'PrecipTotal_60', 'Tavg_14', 'Tavg_28', 'Tavg_60', 'Tmin_14', 'Tmin_28',
       'Tmin_60', 'Tmax_14', 'Tmax_28', 'Tmax_60', 'DewPoint_14',
       'DewPoint_28', 'DewPoint_60', 'Species_CULEX ERRATICUS',
       'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
       'Species_CULEX RESTUANS', 'Species_CULEX SALINARIUS',
       'Species_CULEX TARSALIS', 'Species_CULEX TERRITANS', 'Month', 'Day',
       'Dist_0', 'Dist_1'],
      dtype='object')

In [43]:
# Features to include in model
features = [
     'Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb',
       'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir',
       'AvgSpeed', 'Rain', 'PrecipTotal_14', 'PrecipTotal_28', 'PrecipTotal_60',
       'Tavg_14', 'Tavg_28', 'Tavg_60', 'Tmin_14', 'Tmin_28', 'Tmin_60',
       'DewPoint_14', 'DewPoint_28','DewPoint_60','Tmax_14','Tmax_28','Tmax_60', 'Species_CULEX ERRATICUS',
       'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
       'Species_CULEX RESTUANS', 'Species_CULEX SALINARIUS',
       'Species_CULEX TARSALIS', 'Species_CULEX TERRITANS', 'Month', 'Day',
       'Dist_0', 'Dist_1']

In [44]:
X = df[features]
y = df['WnvPresent']

### Modeling

In [45]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [46]:
rfc = RandomForestClassifier(class_weight='balanced', max_features='sqrt', min_samples_leaf=5, n_estimators=1000, n_jobs=-1)
rfc.fit(X, y)


RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=5,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=1000, n_jobs=-1, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [47]:
feature_import = rfc.feature_importances_
pd.DataFrame(columns=['feature', 'importance'], 
             data=list(zip(features, feature_import))).sort_values('importance',ascending=False).head(10)


,feature,importance
36,Dist_0,0.145145
37,Dist_1,0.139892
35,Day,0.084725
23,DewPoint_60,0.060516
20,Tmin_60,0.055777
17,Tavg_60,0.044041
34,Month,0.042500
22,DewPoint_28,0.030401
16,Tavg_28,0.027195
29,Species_CULEX PIPIENS/RESTUANS,0.024177


# Test export

In [48]:
test['Station'] = [closest_point(x) for x in test[['Latitude','Longitude']].values]
test['Date'] = pd.to_datetime(test['Date'])
test = test.merge(weather, on=['Date','Station'])

In [49]:
test = pd.get_dummies(test, columns=['Species'])

In [50]:
test['Month'] = test['Date'].dt.month
test["Day"] = test['Date'].dt.dayofyear

In [51]:
# Repeat for test data
for i,area in enumerate(wnvpresent_lat_lon):
    test['Dist_' + str(i)] = [haversine(row, (area),unit='mi') for row in zip(test['Latitude'],test['Longitude'])]

In [52]:
# Make match above
X_test = test[features]


In [53]:
# Whatever model you decided on:
predictions = rfc.predict(X_test)

In [54]:
submission = pd.DataFrame(columns=['Id', 'WnvPresent'], data=list(zip(test['Id'], predictions)))
submission = submission.set_index('Id')
submission['WnvPresent'].value_counts()

0    115681
1       612
Name: WnvPresent, dtype: int64

In [55]:
predictions = rfc.predict_proba(X_test)

In [56]:
predictions = rfc.predict_proba(X_test)
submission = pd.DataFrame(columns=['Id', 'WnvPresent'], data=list(zip(test['Id'], predictions)))

In [57]:
submission.WnvPresent = submission['WnvPresent'].map(lambda x: x[1])


In [58]:
submission.to_csv('submission.csv',index=False)

In [59]:
submission.head()

,Id,WnvPresent
0,1,0.014477
1,2,0.026707
2,3,0.022091
3,4,0.022288
4,5,0.022288
